## L'ÎLE - Infocentre littéraire des écrivains québécois

Centre de documentation virtuel qui a pour mission de promouvoir la littérature québécoise. Il s’adresse aux étudiants, aux chercheurs, aux journalistes et à tous ceux qui s’intéressent aux écrivains québécois et à la littérature québécoise.

Regroupe des informations sur des écrivains québécois ou canadiens d’expression française, auteurs d’au moins deux livres d’au moins 48 pages, publiés par une maison d’édition reconnue, dont les livres appartiennent à l’un ou l’autre des genres suivants :
* le roman
* le récit
* la nouvelle
* le conte
* la poésie
* le théâtre
* l’essai ou l’étude
* la bande dessinée
* la biographie
* la littérature jeunesse


Le site fournit aussi une très grande liste à des sites externes (librairies, maisons d'édition, sites personnels des auteurs, associations, etc.)

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import *
from bs4 import BeautifulSoup
import re
import pandas as pd
import os

In [3]:
class Auteur:
    def __init__(self, nom = ""):
        self.nom = nom
        self.uri = ""
        self.bio = ""
        self.genres = []
        self.site = ""
        self.pseudonyme = ""

class Oeuvre:
    def __init__(self, id, titre = " "):
        self.id = id
        self.titre = titre
        self.auteurs = []
        self.lieuPublication = ""
        self.editeur = ""
        self.annee = ""
        self.isbn = ""
        

In [4]:
# Fonctions utiles

def extraireInfoOeuvre(id, contenu):
    oeuvre = Oeuvre(id)
#     print(contenu)

    
    # On extrait l'année et le titre
    annee = re.search(' \([0-9,-|]+\)',contenu[0])
    if annee:
        oeuvre.annee = annee.group(0)[2:-1]
        oeuvre.titre = contenu[0].replace(annee.group(0),'')

    else:
        oeuvre.titre = contenu[0]
        print('OUPS (pas d\'annéee dans le titre):',contenu[0])
               

    # On extrait le lieu de publication et l'éditeur
    # on cherche d'abord la chaîne qui commence par ", "
    edition = ""
    for c in contenu:
        if c.startswith(', '):
            edition = c
            break
    if edition: 
        edition = edition[2:]
        if ':' not in edition:
            print('OUPS (pas de lieu de publication):',edition)
            oeuvre.editeur = edition.strip()
        else:
            edition_infos = edition.split(':')
            oeuvre.lieuPublication = edition_infos[0].strip() 
            oeuvre.editeur = edition_infos[1].strip()
            
    else:
        print('OUPS (pas d\'édition spécifiée):', edition)

    for c in contenu:
        if c.strip().startswith('ISBN :'):
            oeuvre.isbn = c.strip()[7:]

    return oeuvre

def traiter_bio(texte):
    if texte.startswith('Notice biographique\n'):
        return texte[20:]
    else:
        return texte

def traiter_lieu_publication(texte):
    texte = texte.strip()
    if len(texte) > 2 and texte[0] == '[' and texte[-1] == ']':
        return texte[1:-1]
    else:
        return texte
    
   

# Extractions des hyperliens sur les auteurs de ILE

In [5]:
#launch url
url = "http://recif.litterature.org/recherche/"

# create a new Firefox session
driver = webdriver.Chrome()
#driver.implicitly_wait(30)
driver.get(url)


In [6]:
bouton = driver.find_element_by_class_name('btn_chercher')
bouton.click()

liens_auteurs = []
for a in driver.find_elements_by_css_selector('#colonne_resultats > dl > dd > ul > li > a '):
    liens_auteurs.append(a.get_attribute('href'))


In [7]:
print(len(liens_auteurs))

1791


# Extraction des infos sur les auteurs et leurs oeuvres

In [8]:
auteurs = {}
oeuvres = {}



for a in liens_auteurs:
    auteur = Auteur()
    driver.get(a)

    # On extrait l'URI
    auteur.uri = a 

    # On extrait le nom de l'auteur
    auteur.nom = driver.find_element_by_css_selector('#ecrivain .desc > h1').text

    # On extrait la notice biographique
    try:
        bio = driver.find_element_by_css_selector('body div#wrapper div#contenu div#bio ') 
        suite = bio.find_element_by_css_selector('a.suite')
        if suite:
            try:
                suite.click()
            except ElementClickInterceptedException:
                pass
        auteur.bio = bio.text
    except NoSuchElementException:
        print('Cet auteur n\'a pas de bio:',auteur.nom)
        pass

    # On extrait les infos de la fiche
    fiche = driver.find_element_by_css_selector('div#contenu div#fiche') 
    items = fiche.find_elements_by_css_selector('div.desc :nth-child(2) > dt')

    position = 0
    print(auteur.nom)
    for i in items:
#         print(i.text)
        position += 1
        if i.text == "Genre(s) littéraire(s) :":
            genres = fiche.find_elements_by_css_selector('div.desc :nth-child(2) > dd:nth-of-type({0}) > a'.format(position))
            for g in genres:
#                 print(g.text)
                auteur.genres.append(g.text)
        if i.text == "Site Web :":
            site = fiche.find_element_by_css_selector('div.desc :nth-child(2) > dd:nth-of-type({0}) > a'.format(position))
#             print(site.text)  
            auteur.site = site.text
        if i.text == "Pseudo :":
            pseudo = fiche.find_element_by_css_selector('div.desc :nth-child(2) > dd:nth-of-type({0})'.format(position))
#             print(pseudo.text.strip())  
            auteur.pseudonyme = pseudo.text.strip()           
  

    liste_oeuvres = driver.find_elements_by_css_selector('div#oeuvres li')
    print(len(liste_oeuvres),'oeuvres')
    for o in liste_oeuvres:
        content = list(BeautifulSoup(o.get_property('innerHTML')).stripped_strings)
        oeuvre = extraireInfoOeuvre(o.get_attribute('id')[6:], content)

        # On ajoute le nom de l'auteur (dans la description de l'oeuvre, il se peut qu'il y ait 
        # d'autres auteurs, mais on les ignore pour l'instant)
        oeuvre.auteurs = [auteur.uri]
        oeuvres[oeuvre.id] = oeuvre
#         print(oeuvre.id)
#         print(oeuvre.titre)
        if len(oeuvre.annee) > 6:
            print(oeuvre.annee)

#         print(oeuvre.annee)
#         print(oeuvre.auteurs)
#         print(oeuvre.editeur)
#         print(oeuvre.isbn)
#         print('*****')
    print('---')

    auteurs[a] = auteur





., Nathanaël
20 oeuvres
---
Acquelin, José
37 oeuvres
---
Adam, Monique
3 oeuvres
---
Agnant, Marie-Célie
19 oeuvres
---
Ajoul, Hassan
1 oeuvres
---
Al-Yasiri, Issa Hassan
23 oeuvres
OUPS (pas de lieu de publication): London, 2011
OUPS (pas de lieu de publication): Lebanon, 2018
OUPS (pas de lieu de publication): London, 2012
OUPS (pas de lieu de publication): Madrid, 2018
OUPS (pas de lieu de publication): Montréal, 2015
---
Alain, Sonia
17 oeuvres
---
Alarie, Donald
31 oeuvres
---
Alavo, Yves
6 oeuvres
---
Albert, Michel
7 oeuvres
---
Allard, Francine
61 oeuvres
---
Allard, Jacques
12 oeuvres
---
Allen, Michelle
6 oeuvres
---
Almeyda Morales, Miguel
7 oeuvres
---
Alonzo, Anne-Marie
23 oeuvres
---
Amabili-Rivet, Rita
16 oeuvres
OUPS (pas de lieu de publication): Éditions Universitaires Européennes, 2012
OUPS (pas de lieu de publication): Éditions Guido Amabili, 2010
OUPS (pas de lieu de publication): Seconda Edizione, Librati Editrice, 2012
OUPS (pas de lieu de publication): Edarc Edi

Benfares, Mostafa
4 oeuvres
---
Benjamin, Franz
7 oeuvres
---
Benkirane, Kamal
5 oeuvres
---
Benoit, Jacques
19 oeuvres
---
Benoit, Réal
7 oeuvres
---
Bergeron, Alain M.
178 oeuvres
---
Bergeron, Claire
9 oeuvres
---
Bergeron, Denys
16 oeuvres
---
Bergeron, Diane
15 oeuvres
---
Bergeron, Dominique
7 oeuvres
---
Bergeron, Léandre
21 oeuvres
1981?,1980
OUPS (pas de lieu de publication): [Montréal] Editions québécoises, 1971, v. (en majeure part. ill.) 28 cm.
OUPS (pas de lieu de publication): [Montréal] Editions Québécoises, 1970, 207 p. ill. carte 18 cm.
OUPS (pas de lieu de publication): [Montréal Editions québécoise, 1972, 185p. ill. 19cm.
OUPS (pas d'annéee dans le titre): The Québécois dictionary
---
Bergeron, Lucie
42 oeuvres
---
Bergeron, Mario
12 oeuvres
---
Bernard, Marie Christine
8 oeuvres
---
Bernard, Michèle
2 oeuvres
---
Bernard-Lenoir, Anne
20 oeuvres
---
Bernèche, Jean
13 oeuvres
---
Bernier, Jovette-Alice
12 oeuvres
1944-,1950
---
Bernier, Mélina
1 oeuvres
---
Berryl, Ro

Brouillet, Chrystine
114 oeuvres
1994|1995
1991,1990
1991-1994
1990-1994
---
Brouillette, Marc André
7 oeuvres
---
Brousseau, Linda
7 oeuvres
---
Brousseau, Marilou
6 oeuvres
---
Brulotte, Gaëtan
27 oeuvres
OUPS (pas de lieu de publication): Editions Trois-Pistoles, Coll. «Écrire», 2003, 180 p.
---
Brun (Gauthier), Huguette
2 oeuvres
OUPS (pas de lieu de publication): Éditions Vaudreuil, 2010
---
Brunelle, Joséane
3 oeuvres
---
Brunet, Francine
3 oeuvres
---
Buies, Arthur
40 oeuvres
1840-1901
1986|1991
1873|1875
1855-1901
1868|1869
1864|1867
---
Bujold, Michel
10 oeuvres
OUPS (pas de lieu de publication): Nuit Balnche éditeurs, 1998
---
Bujold, Réal-Gabriel
32 oeuvres
OUPS (pas de lieu de publication): Lyrelou, 2006
OUPS (pas de lieu de publication): Aigle moqueur éditeur, 2006
---
Bureau, Ginette
20 oeuvres
OUPS (pas de lieu de publication): BalboaPress, 2016, 166 pages.
---
Bureau, Luc
13 oeuvres
---
Bussières, Simone
33 oeuvres
1968?,1967
1968,1967
OUPS (pas de lieu de publication):

---
Cet auteur n'a pas de bio: Côté, Diane-Jocelyne
Côté, Diane-Jocelyne
7 oeuvres
---
Côté, Jacques
16 oeuvres
---
Côté, Johanne Alice
4 oeuvres
---
Côté, Laurier
7 oeuvres
OUPS (pas de lieu de publication): Éditions audiovisuelles, 1983
---
Côté, Michel
13 oeuvres
---
Côté, Reine-Aimée
5 oeuvres
---
Côtes, Gilles
15 oeuvres
---
Cotnoir, Louise
14 oeuvres
---
Cotten, Sonia
4 oeuvres
---
Couët, Bernard
5 oeuvres
---
Coulet, Sylvie
4 oeuvres
---
Courchesne, Luc C.
6 oeuvres
---
Courtemanche, Gil
27 oeuvres
---
Cousineau, Mélanie
7 oeuvres
---
Cousture, Arlette
29 oeuvres
1996,1994
OUPS (pas d'annéee dans le titre): Ces enfants d'ailleurs (1994, 1992)
1993|1995
1997,1995
1995,1997
OUPS (pas d'annéee dans le titre): Les Filles de Caleb (1988, 1989)
1986|1987
OUPS (pas d'annéee dans le titre): Les Filles de Caleb (1985, 1986)
OUPS (pas d'annéee dans le titre): Les Filles de Caleb (1985, 1986)
---
Couture, Lucien
4 oeuvres
---
Couture, Maryse
1 oeuvres
---
Couture, Yvon H.
5 oeuvres
OUPS (p

Dotrice, Pierrette
6 oeuvres
---
Doyon, Paule
61 oeuvres
OUPS (pas de lieu de publication): Shawinigan, 2002
OUPS (pas de lieu de publication): Shawinigan, 2002
OUPS (pas de lieu de publication): Shawinigan, 2002
OUPS (pas de lieu de publication): Shawinigan, 2002
OUPS (pas de lieu de publication): Shawinigan, 2002
OUPS (pas de lieu de publication): Shawinigan, 2002
OUPS (pas de lieu de publication): [Sherbrooke] Editions Paulines, Contes du chalet bleu ; 6, 1971, 14p. ill. en coul. 25cm.
OUPS (pas de lieu de publication): [Sherbrooke] Editions Paulines, Contes du chalet bleu ; 3, 1971, 14p. ill. (part. en coul.) 25cm.
OUPS (pas de lieu de publication): [Sherbrooke] Editions Paulines, Contes du chalet bleu ; 21, 1972, 14p. ill. (part. en coul.) 25cm.
---
Cet auteur n'a pas de bio: Drapeau, Renée-Berthe
Drapeau, Renée-Berthe
2 oeuvres
---
Drouin, Marisol
2 oeuvres
---
Drouin, Véronique
16 oeuvres
---
Dubé, Danielle
9 oeuvres
---
Dubé, Frédérique
1 oeuvres
---
Dubé, Jasmine
35 oeuvres
OU

...

Monet-Chartrand, Simonne
5 oeuvres
---
Monette, Denis
48 oeuvres
---
Monette, Hélène
19 oeuvres
---
Monette, Madeleine
26 oeuvres
OUPS (pas de lieu de publication): France, 2018, Collection patrimoniale de livres d'artistes de BAnQ, Montréal.
OUPS (pas de lieu de publication): France, 2018, Collection patrimoniale de livres d'artistes de BAnQ, Montréal.
---
Cet auteur n'a pas de bio: Monette, Pierre
Monette, Pierre
9 oeuvres
OUPS (pas d'annéee dans le titre): Le guide du tango
---
Mongeau, France
12 oeuvres
---
Mongeau, Paule
6 oeuvres
---
Mongeau, Serge
30 oeuvres
1967|1970
---
Mongrain, Serge
20 oeuvres
---
Montescu, Cristina
6 oeuvres
---
Monticone, Ugo
9 oeuvres
OUPS (pas de lieu de publication): Éditions du CRAM, 2005
---
Montour, Nancy
21 oeuvres
---
Montpetit, Charles
12 oeuvres
---
Montpetit, Marie-Hélène
3 oeuvres
---
Morency, Isabelle
7 oeuvres
---
Morency, Joanne
7 oeuvres
---
Morency, Pierre
30 oeuvres
---
Moreno, Caroline
6 oeuvres
---
Morin, Claude
10 oeuvres
OUPS (pas de

Perron, Jean
19 oeuvres
OUPS (pas de lieu de publication): 1987
---
Petitpas, Louise
1 oeuvres
---
Petits, Jean-Pierre
3 oeuvres
---
Petrowski, Minou
5 oeuvres
---
Petrowski, Nathalie
7 oeuvres
---
Peyrouse, Anne
4 oeuvres
---
Phelps, Anthony
29 oeuvres
OUPS (pas de lieu de publication): in Callaloo, 15, 2, (spring 1992), 1987, pp 347-380.
OUPS (pas de lieu de publication): in Poet and Critic, 3, 16, 1985
---
Piazza, François
10 oeuvres
OUPS (pas de lieu de publication): Éditions du Mémorial, tome 4, 1981
OUPS (pas de lieu de publication): Éditions du Mémorial, tome 3, 1980
---
Picard, Gaëtan
4 oeuvres
---
Picard-Sioui, Louis-Karl
10 oeuvres
---
Piché, Alphonse
15 oeuvres
---
Piché, Leslie
1 oeuvres
---
Pierre, Émeline
3 oeuvres
---
Pigeon, Daniel
7 oeuvres
---
Pilon, Jean-Guy
10 oeuvres
1969,1968
---
Pilon-Delorme, Lise-Anne
6 oeuvres
---
Pilote, Arlette
18 oeuvres
2000-2001
---
Piotte, Jean-Marc
24 oeuvres
---
Pipar, Rosette
8 oeuvres
---
Pitre, Isabelle
2 oeuvres
---
Piuze, Simone
7

Rioux, Billy
7 oeuvres
---
Rioux, François
2 oeuvres
---
Rioux, Hélène
30 oeuvres
---
Rioux, Marcel
32 oeuvres
1970,1969
---
Rivard, Élise
1 oeuvres
---
Rivard, Yvon
16 oeuvres
---
Riverin, Agnès
4 oeuvres
---
Rivet, Mélanie
7 oeuvres
OUPS (pas de lieu de publication): 2015
OUPS (pas de lieu de publication): Grand Théâtre de Québec, 2015
OUPS (pas de lieu de publication): Théâtre Hector-Charland, L’Assomption, 2004
OUPS (pas de lieu de publication): Théâtre Arts Côte à Côte, Mascouche, 2009
---
Rivière, Sylvain
81 oeuvres
OUPS (pas de lieu de publication): Éditions Humanitas, 1992, coll. Nouvelle optique.
OUPS (pas de lieu de publication): Éditions Humanitas, 1993
OUPS (pas de lieu de publication): Éditions Humanitas, 1997
OUPS (pas de lieu de publication): Éditions Trois-Pistoles, 1997
OUPS (pas de lieu de publication): Éditions Humanitas, 1992
OUPS (pas de lieu de publication): Éditions Humanitas, 1992
OUPS (pas de lieu de publication): Éditions Humanitas, 1995
OUPS (pas de lieu de p

Simpson, Danièle
28 oeuvres
1994-1995
1988,1987
1988,1987
---
Sloate, Daniel
12 oeuvres
1975-1981
1986,1985
OUPS (pas de lieu de publication): [Montréal Editions Maisonneuve, 1972, 140p. 21 cm.
---
Smith, Neil
4 oeuvres
---
Somain, Jean-François
54 oeuvres
1984,1983
1990,1989
---
Souaid, Carolyn Marie
6 oeuvres
---
Soucy, Erika
4 oeuvres
---
Soucy, Gaétan
18 oeuvres
---
Soucy, Geneviève
3 oeuvres
---
Soucy, Jean-Yves
26 oeuvres
---
Soudeyns, Maurice
18 oeuvres
OUPS (pas de lieu de publication): compte d'auteur, 1973
---
Soulières, Robert
66 oeuvres
OUPS (pas de lieu de publication): Éditions Médiaspaul, 1995, 263 p. ; coll. Lectures VIP.
OUPS (pas de lieu de publication): Éditions Les 400 coups, 1998
---
St-Georges, Madeleine
5 oeuvres
---
St-Louis, Yvon
4 oeuvres
OUPS (pas d'annéee dans le titre): Services publics
OUPS (pas de lieu de publication): Trafford Publishing, 2006, 214 p., 14 x 1.2 x 21.6 cm.
---
Stanké, Alain
36 oeuvres
---
Stanké, Claudie
19 oeuvres
---
Stanton, Julie
16 o

Vigneau, Claire
3 oeuvres
---
Cet auteur n'a pas de bio: Vigneault, François
Vigneault, François
7 oeuvres
---
Vigneault, Gilles
74 oeuvres
1961|1965
1959|1964
1971,1965
---
Vigneault, Guillaume
6 oeuvres
---
Vigneault, Robert
12 oeuvres
1960-1985
---
Villemaire, Yolande
43 oeuvres
OUPS (pas de lieu de publication): WIP Édizioni, 2007
1988,1987
1984,1980
---
Villeneuve, Félix
2 oeuvres
---
Villeneuve, Marie-Paule
6 oeuvres
---
Villeneuve, Marité
9 oeuvres
---
Villeneuve, Mathieu
1 oeuvres
---
Villeneuve, Mireille
33 oeuvres
---
Vincelette, Isabelle
1 oeuvres
---
Vincent, Marie-Christine
5 oeuvres
---
Vincent, Pauline
9 oeuvres
---
Vivier, Karine
1 oeuvres
---
Vonarburg, Élisabeth
53 oeuvres
OUPS (pas d'annéee dans le titre): La tapisserie de Fionavar
OUPS (pas d'annéee dans le titre): Le dernier rayon du soleil
OUPS (pas d'annéee dans le titre): Les lions d'Al-Rassan
OUPS (pas d'annéee dans le titre): Ysabel
OUPS (pas d'annéee dans le titre): La tapisserie de Fionavar
OUPS (pas d'année

In [11]:
print(len(liens_auteurs))
print(len(auteurs), 'auteurs')
print(len(oeuvres), 'oeuvres')

# for a in auteurs.values():
#     print(a.uri)
# driver.get('http://recif.litterature.org/recherche/ecrivains/.-nathanael-825/')
# page_auteur = driver.find_element_by_id('bio')
# print(page_auteur.text)

1791
1791 auteurs
27109 oeuvres


In [13]:

f = open('../Data/ILE/auteurs_ILE.csv',"w")
for a in auteurs.values():
    f.write("{}<>{}<>".format(a.uri, a.nom))
    f.write(",".join(a.genres) + "<>")
    f.write("{}<>{}<>{}".format(a.pseudonyme if a.pseudonyme else " " , a.site if a.site else " ", a.bio))
    f.write("\n-----\n")
f.close()

f = open('../Data/ILE/oeuvres_ILE.csv',"w")
for o in oeuvres.values():
    f.write("{}<>{}<>{}<>".format(o.id, o.titre.replace('\n',' '), o.annee))
    f.write(",".join(o.auteurs) + "<>")
    f.write("{}<>{}<>{}".format(o.editeur.replace('\n',' '), o.lieuPublication, o.isbn))
    f.write("\n")
f.close()